## read data from files

In [28]:
import janitor
import os
import glob
#import mitosheet
import ipywidgets as widgets
%matplotlib inline
#%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

from sklearn.model_selection import cross_val_score


%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

In [29]:
#!pip install sklearn

In [30]:
import pandas as pd
p=0.50
y=pd.read_csv("y_219_25mai.csv", parse_dates=["date"], index_col="date")
X=pd.read_csv("X_219_25mai.csv", parse_dates=["date"], index_col="date")
#y = y.values.ravel()
# X_train = X[:int(X.shape[0]*p)]
# X_test = X[int(X.shape[0]*p):]
# y_train = y[:int(X.shape[0]*p)]
# y_test = y[int(X.shape[0]*p):]
# X_train.shape

In [31]:
period = "30T"
max_ = X.resample(period).max()
max_.columns +="_max"
min_ = X.resample(period).min()
min_.columns +="_min"
mean_ = X.resample(period).mean()
mean_.columns +="_mean"
std_ = X.resample(period).std()
std_.columns +="_std"
sum_ = X.resample(period).sum()
sum_.columns +="_sum"
dfs=[max_,min_,mean_,std_, sum_]

In [32]:
y = y.resample(period).mean()
# y1 =y1.values.ravel()

In [33]:
import functools 
df = functools.reduce(lambda left,right: pd.merge(left,right,on='date'), dfs)

In [34]:
X = df.copy()

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.35, shuffle=False)

In [65]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False) 

RandomForestRegressor()

In [22]:
y_train=y_train.values.ravel()
rfr.fit(X_train, y_train)
score = rfr.score(X_train, y_train)
print("R-squared:", score) 

R-squared: 0.9675747198471185


In [23]:
from sklearn.metrics import mean_squared_error
y_pred = rfr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred,squared=False)
print("MSE: ", mse)
print("RMSE: ", rmse) 

MSE:  4409.079463773093
RMSE:  66.40089957051104


In [24]:
from sklearn.feature_selection import SelectFromModel
# rfr = clf.fit(xtrain, ytrain)
# clf.feature_importances_  
# array([ 0.04...,  0.05...,  0.4...,  0.4...])
model = SelectFromModel(rfr, prefit=True)
X_new = model.transform(X_train)
X_new 

array([[ 119.33333333,   24.48666667,  102.33333333, ...,   33.        ,
          68.        , 3071.7       ],
       [ 372.66666667,   27.24      ,  360.33333333, ...,   66.        ,
         148.        , 6143.65      ],
       [ 451.        ,   27.26      ,  440.        , ...,   65.75      ,
         174.5       , 6144.25      ],
       ...,
       [ 286.75      ,   24.014     ,  332.5       , ...,   23.6       ,
         472.        , 6085.5       ],
       [ 292.        ,   24.005     ,  342.        , ...,   23.        ,
         479.        , 6084.        ],
       [ 297.25      ,   24.0025    ,  344.2       , ...,   21.68333333,
         477.        , 6082.9       ]])

In [25]:
X_train.loc[:,model.get_support()].columns

Index(['tvoc_100_max', 'temperature_100_max', 'tvoc_101_max',
       'temperature_101_max', 'sound_101_max', 'sound_104_max', 'tvoc_106_max',
       'sound_107_max', 'sound_108_max', 'co2_110_max', 'light_111_max',
       'weather_hum_max', 'weather_bar__max', 'weather_rad__max',
       'temperature_103_min', 'temperature_104_min', 'sound_108_min',
       'light_111_min', 'weather_hum_min', 'weather_bar__min',
       'weather_rad__min', 'tvoc_101_mean', 'temperature_104_mean',
       'sound_108_mean', 'weather_out_mean', 'weather_hum_mean',
       'weather_bar__mean', 'co2_100_std', 'temperature_100_std',
       'humidity_100_std', 'tvoc_101_std', 'temperature_101_std',
       'sound_102_std', 'light_103_std', 'sound_104_std', 'tvoc_106_std',
       'temperature_108_std', 'co2_111_std', 'temperature_112_std',
       'temperature_103_sum', 'sound_108_sum', 'weather_hum_sum',
       'weather_bar__sum'],
      dtype='object')

array([-78.49519808])

In [80]:
y1

array([ -83.08654302,  -86.87929348,  143.64879825,   53.11767063,
       -137.12562568,  -76.17911331,  120.97431801, -152.98264176,
         92.4214761 , -178.17606253,  255.89033696,  109.30931948,
        -25.14676048,  122.48056043,  -63.43119168,   63.1024734 ,
        170.08532165,  298.62248374,  147.08015727,   13.98238816,
        -63.20215872, -160.3243681 ,   98.48277148,  -64.15689374,
         89.44793595, -199.88331975, -235.11439276,  -92.80330438,
         65.97137792, -240.35957012,   58.53672285,   17.64118695,
         39.19532947, -159.67494116,  214.40536597, -115.37493987,
         17.39566243,   17.03944817, -190.71934088, -253.19313503,
        152.34316164,   45.80067497,   94.40807603,    3.99305466,
        -13.84420769, -151.37817847, -104.29954489, -219.2608145 ,
        -30.42886711, -162.87549183, -103.4805689 ,  -25.85948739,
       -140.55707091,   22.21891754,  -26.26358559,   25.29018877,
         -4.88580563, -156.20005888,   57.62585988,    4.61951

In [37]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso

In [38]:
pipeline = Pipeline([
                     ('scaler',StandardScaler()),
                     ('model',Lasso())
])

In [81]:
search = GridSearchCV(pipeline,
                      {'model__alpha':np.arange(0.1,60,0.1)},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3
                      )

In [82]:
import warnings
warnings.filterwarnings("ignore")
search.fit(X_train,y_train)

Fitting 5 folds for each of 599 candidates, totalling 2995 fits
[CV 1/5] END .............model__alpha=0.1;, score=-60247.440 total time=   0.1s
[CV 2/5] END .............model__alpha=0.1;, score=-15533.473 total time=   0.1s
[CV 3/5] END .............model__alpha=0.1;, score=-43831.047 total time=   0.1s
[CV 4/5] END .............model__alpha=0.1;, score=-27899.449 total time=   0.1s
[CV 5/5] END .............model__alpha=0.1;, score=-24213.550 total time=   0.1s
[CV 1/5] END .............model__alpha=0.2;, score=-20498.408 total time=   0.1s
[CV 2/5] END .............model__alpha=0.2;, score=-14530.806 total time=   0.1s
[CV 3/5] END .............model__alpha=0.2;, score=-50368.777 total time=   0.1s
[CV 4/5] END .............model__alpha=0.2;, score=-24664.658 total time=   0.1s
[CV 5/5] END .............model__alpha=0.2;, score=-22112.854 total time=   0.1s
[CV 1/5] END model__alpha=0.30000000000000004;, score=-54152.776 total time=   0.0s
[CV 2/5] END model__alpha=0.30000000000000

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', Lasso())]),
             param_grid={'model__alpha': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,
        1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,
        2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,  3.3,
        3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,
        4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,  5.5,
        5.6,  5.7,  5...
       52.9, 53. , 53.1, 53.2, 53.3, 53.4, 53.5, 53.6, 53.7, 53.8, 53.9,
       54. , 54.1, 54.2, 54.3, 54.4, 54.5, 54.6, 54.7, 54.8, 54.9, 55. ,
       55.1, 55.2, 55.3, 55.4, 55.5, 55.6, 55.7, 55.8, 55.9, 56. , 56.1,
       56.2, 56.3, 56.4, 56.5, 56.6, 56.7, 56.8, 56.9, 57. , 57.1, 57.2,
       57.3, 57.4, 57.5, 57.6, 57.7, 57.8, 57.9, 58. , 58.1, 58.2, 58.3,
       58.4, 58.5, 58.6, 58.7, 58.8, 58.9, 59. , 59.1, 

In [55]:
search.best_params_

{'model__alpha': 50}

In [56]:
coefficients = search.best_estimator_.named_steps['model'].coef_

In [57]:
importance = np.abs(coefficients)

In [58]:
# importance

In [59]:
features = X_train.columns
np.array(features)[importance > 0]

array(['temperature_100_max', 'temperature_105_max', 'co2_111_max',
       'tvoc_106_std'], dtype=object)

In [ ]:
X1 = X['temperature_100_max', 'temperature_105_max', 'co2_111_max',
       'tvoc_106_std']

In [63]:
selectedFeatures = ['temperature_100_max', 'temperature_105_max', 'co2_111_max',
       'tvoc_106_std']
X_test_selected = X_test[selectedFeatures].copy()
X_train_selected = X_train[selectedFeatures].copy()

In [66]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False) 

RandomForestRegressor()

In [68]:
rfr.fit(X_train_selected, y_train)
score = rfr.score(X_train_selected, y_train)
print("R-squared:", score) 

R-squared: 0.9479690129383113


In [70]:
from sklearn.metrics import mean_squared_error
y_pred = rfr.predict(X_test_selected)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred,squared=False)
print("MSE: ", mse)
print("RMSE: ", rmse) 

MSE:  4175.5474709455775
RMSE:  64.61847623509532


In [77]:
y_pred.max()

922.6738799999995

In [64]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

X2 = sm.add_constant(X_train_selected)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.501
Model:                            OLS   Adj. R-squared:                  0.499
Method:                 Least Squares   F-statistic:                     306.1
Date:                Mon, 30 May 2022   Prob (F-statistic):          2.19e-182
Time:                        12:09:43   Log-Likelihood:                -7446.6
No. Observations:                1226   AIC:                         1.490e+04
Df Residuals:                    1221   BIC:                         1.493e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const               -1010.1530    

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=10, step=1)
selector = selector.fit(X, y)
selector.support_
selector.ranking_

In [ ]:
selector.ranking_

In [ ]:
plt.plot(y_train)
plt.show()

In [ ]:
# from sklearn.datasets import make_friedman1
# from sklearn.feature_selection import RFECV
# from sklearn.svm import SVR
# #X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
# estimator = SVR(kernel="linear")
# selector = RFECV(estimator, step=1, cv=5)
# selector = selector.fit(X1, y1)
# selector.support_

# selector.ranking_

In [ ]:
# from sklearn.datasets import make_friedman1
# from sklearn.feature_selection import RFE
# from sklearn.svm import SVR
# # X, y = make_friedman1(n_samples=50, n_features=10, random_state=0)
# estimator = SVR(kernel="linear")
# selectorRFE = RFE(estimator, n_features_to_select=10, step=1)
# selectorRFE = selectorRFE.fit(X_train, y_train)
# selectorRFE.support_
# selectorRFE.ranking_

In [ ]:
selectorRFE.feature_names_in_

In [ ]:
colonesSelected = []
for i,j in enumerate(selectorRFE.ranking_):
    # print(j)
    if j<5:
        colonesSelected.append(i)

In [ ]:
var=X_train.columns[colonesSelected]

In [ ]:
var

In [5]:
var =['temperature_103', 'light_103', 'sound_103', 'temperature_104',
       'light_104', 'light_105', 'light_106', 'temperature_107', 'light_107',
       'tvoc_108', 'sound_108', 'temperature_110', 'temperature_112']

In [6]:
import pandas
from sklearn import linear_model

X = X[var].copy()
print(X.columns)
col = [0, 4, 5, 7, 8, 9, 13, 14, 15, 17, 18, 21, 25, 26, 29, 30, 32, 33, 34, 35, 36]
col = [0, 4, 5, 7,  9, 13, 15, 18, 21, 26,  30]
col2    =[0, 1, 2, 3, 4, 5,6,  8, 10, 11, 13, 14,15]
# X= X[col]
#X= X.iloc[:,col]
y = y
# importing train_test_split from sklearn
from sklearn.model_selection import train_test_split
# splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, shuffle = False)

X_train = X_train.values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
datascaler=scaler.fit(X_train)
X_train= datascaler.transform(X_train)
X_test= datascaler.transform(X_test)

regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)


print(regr.coef_) 

Index(['temperature_103', 'light_103', 'sound_103', 'temperature_104',
       'light_104', 'light_105', 'light_106', 'temperature_107', 'light_107',
       'tvoc_108', 'sound_108', 'temperature_110', 'temperature_112'],
      dtype='object')
[[-2.51138950e+01 -6.04513917e+01 -4.23076081e-01  8.23998451e+01
   3.36715547e+01  5.12095935e+00  6.72743693e+02  6.83095055e+01
  -5.31896690e+02  4.32990561e+01 -7.60309891e+00  3.77790832e+00
  -5.02930975e+01]]


In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

X2 = sm.add_constant(X_train)
est = sm.OLS(y_train, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:     elec_general_219_w   R-squared:                       0.804
Model:                            OLS   Adj. R-squared:                  0.804
Method:                 Least Squares   F-statistic:                     2331.
Date:                Fri, 06 May 2022   Prob (F-statistic):               0.00
Time:                        14:38:29   Log-Likelihood:                -46285.
No. Observations:                7392   AIC:                         9.260e+04
Df Residuals:                    7378   BIC:                         9.269e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        434.7110      1.476    294.508      0.0

In [8]:
lm = linear_model.LinearRegression()
X, y=X_train, y_train
lm.fit(X,y)
params = np.append(lm.intercept_,lm.coef_)
predictions = lm.predict(X)

newX = pd.DataFrame({"Constant":np.ones(len(X))}).join(pd.DataFrame(X))
MSE = (sum((y-predictions)**2))/(len(newX)-len(newX.columns))

# Note if you don't want to use a DataFrame replace the two lines above with
# newX = np.append(np.ones((len(X),1)), X, axis=1)
# MSE = (sum((y-predictions)**2))/(len(newX)-len(newX[0]))

var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params/sd_b

p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-len(newX.columns)-1))) for i in ts_b]

sd_b = np.round(sd_b,3)
ts_b = np.round(ts_b,3)
p_values = np.round(p_values,3)
params = np.round(params,4)

myDF3 = pd.DataFrame()
myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["Probabilities"] = [params,sd_b,ts_b,p_values]
print(myDF3)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
col= []
for i,pval in enumerate(myDF3["Probabilities"]):
    if pval<=0.001:
        col.append(i-1)
col   = col[1:] 

In [ ]:
col

In [9]:
# importing r2_score module
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_pred)
print("r2 socre is ",score)
print("mean_sqrd_error is ",mean_squared_error(y_test,y_pred))
print("root_mean_squared error of is ",np.sqrt(mean_squared_error(y_test,y_pred)))

r2 socre is  0.5818755142341248
mean_sqrd_error is  8373.730622540243
root_mean_squared error of is  91.50809047587127


In [10]:
d = {'y_pred': y_pred, 'y_test': y_test}
dfplot= pd.DataFrame(data=d)
dfplot.plot()


ValueError: If using all scalar values, you must pass an index

## Missing data generation

In [ ]:
#!pip install torch
#!pip install wget 
import wget
#wget.download('https://raw.githubusercontent.com/BorisMuzellec/MissingDataOT/master/utils.py')

import numpy as np
import pandas as pd
from utils import *
import torch
import seaborn as sns

In [ ]:
df1 = df.copy()



In [ ]:
df.plot()

In [ ]:
#!pip install category_encoders
import category_encoders as ce
# Get a new clean dataframe
obj_df = df1.select_dtypes(include=['object']).copy()

In [ ]:
df1 = obj_df.replace(x)

In [ ]:
df1.plot()

In [ ]:
# Fix the seed ------------------------------------------------------
np.random.seed(2)

In [ ]:
# Function produce_NA for generating missing values ------------------------------------------------------

def produce_NA(X, p_miss, mecha="MCAR", opt=None, p_obs=None, q=None):
    """
    Generate missing values for specifics missing-data mechanism and proportion of missing values. 
    
    Parameters
    ----------
    X : torch.DoubleTensor or np.ndarray, shape (n, d)
        Data for which missing values will be simulated.
        If a numpy array is provided, it will be converted to a pytorch tensor.
    p_miss : float
        Proportion of missing values to generate for variables which will have missing values.
    mecha : str, 
            Indicates the missing-data mechanism to be used. "MCAR" by default, "MAR", "MNAR" or "MNARsmask"
    opt: str, 
         For mecha = "MNAR", it indicates how the missing-data mechanism is generated: using a logistic regression ("logistic"), quantile censorship ("quantile") or logistic regression for generating a self-masked MNAR mechanism ("selfmasked").
    p_obs : float
            If mecha = "MAR", or mecha = "MNAR" with opt = "logistic" or "quanti", proportion of variables with *no* missing values that will be used for the logistic masking model.
    q : float
        If mecha = "MNAR" and opt = "quanti", quantile level at which the cuts should occur.
    
    Returns
    ----------
    A dictionnary containing:
    'X_init': the initial data matrix.
    'X_incomp': the data with the generated missing values.
    'mask': a matrix indexing the generated missing values.s
    """
    
    to_torch = torch.is_tensor(X) ## output a pytorch tensor, or a numpy array
    if not to_torch:
        X = X.astype(np.float32)
        X = torch.from_numpy(X)
    
    if mecha == "MAR":
        mask = MAR_mask(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "logistic":
        mask = MNAR_mask_logistic(X, p_miss, p_obs).double()
    elif mecha == "MNAR" and opt == "quantile":
        mask = MNAR_mask_quantiles(X, p_miss, q, 1-p_obs).double()
    elif mecha == "MNAR" and opt == "selfmasked":
        mask = MNAR_self_mask_logistic(X, p_miss).double()
    else:
        mask = (torch.rand(X.shape) < p_miss).double()
    
    X_nas = X.clone()
    X_nas[mask.bool()] = np.nan
    
    return {'X_init': X.double(), 'X_incomp': X_nas.double(), 'mask': mask}

In [ ]:
X = df.copy()
shelly_219 =  ['state_AA7BF5','state_AA4706', 'state_AA9130', 'state_483FDA81E0F4']
for shelly in shelly_219:
    X = X.drop([shelly ], axis=1)

In [ ]:
X.columns

In [ ]:
X = X.values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
datascaler=scaler.fit(X)
X= datascaler.transform(X)

In [ ]:
import pandas as pd
df= pd.DataFrame(X)
df.index=data.index

In [ ]:
df.head()

In [ ]:
#df.to_csv('dataComplet_219.csv')

## MCAR Generation

In [ ]:
# Minimal example for generating missing data ------------------------
p='5.csv'
X_miss_mcar = produce_NA(X, p_miss=0.05, mecha="MCAR")
X_mcar = X_miss_mcar['X_incomp']
R_mcar = X_miss_mcar['mask']

print("Percentage of newly generated missing values: ", (R_mcar.sum()).numpy()/np.prod(R_mcar.size())*100, " %")

df = X_mcar
df = df.numpy()
mask=R_mcar
mask = mask.numpy()

df = pd.DataFrame(data=df, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])
mask = pd.DataFrame(data=mask, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])

df['time']= data.index
df.set_index('time',inplace = True)
mask['time']= data.index
mask.set_index('time',inplace = True)
df.to_csv('MCAR'+p)
mask.to_csv('mask'+p)

In [ ]:
#df.to_csv('MCAR25.csv')
# supprimer d'abord les deux lignes dans le fichier d'export Neatatmo
index_col="time"
df=pd.read_csv('MCAR40.csv', sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=pd.read_csv('mask40.csv', sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=mask.values


In [ ]:
# Sample mar data -----------------------------------------
X_miss_mar = produce_NA(X, p_miss=0.4, mecha="MAR", p_obs=0.4)

X_mar = X_miss_mar['X_incomp']
R_mar = X_miss_mar['mask']

print("Percentage of generated missing values: ", (R_mar.sum()).numpy()/np.prod(R_mar.size())*100, " %")

ax = sns.heatmap(X_mar, mask=R_mar.numpy()==1, linewidths=0.02, linecolor='black')

df = X_mar
df = df.numpy()
mask_mar = R_mar
mask_mar = mask_mar.numpy()

df = pd.DataFrame(data=df, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])
mask_mar = pd.DataFrame(data=mask_mar, columns=['Temperature', 'Humidity', 'CO2', 'Noise','Pressure'])



In [ ]:
# df['time']= data.index
# df.set_index('time',inplace = True)
# mask_mar['time'] = data.index
# mask_mar.set_index('time',inplace = True)
# df.to_csv('MAR'+p)
# mask_mar.to_csv('mask_mar'+p)

In [ ]:
# Minimal example for generating missing data ------------------------
p='5.csv'

In [ ]:
pourcentage=5
p=